# Error Analysis

*Author: Alex Lewandowski; Alaska Satellite Facility*

This notebook assumes that you have a completed MintPy time series analysis.

---
## 0. Import Required Software



In [ ]:
from pathlib import Path
import sys

from ipyfilechooser import FileChooser
import ipywidgets as widgets
from ipywidgets import Layout
import opensarlab_lib as osl

current = Path("..").resolve()
sys.path.append(str(current))
import util.gps as gps
import util.util as util

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [ ]:
path = Path.home()
fc = FileChooser(path)
print("Select your project's MintPy directory:")
display(fc)

In [ ]:
config_path = Path(fc.selected)
mint_path = config_path.parent
plot_path = mint_path / "plots"

## 1. Spatial Coherence

Spatial Coherence measures phase consistency across neighborhoods of pixels in each interferogram in the stack. A high level of spatial coherence shows that there was little phase change across adjacent pixels. 

In [ ]:
%matplotlib widget
scp_args = f"{mint_path}/avgSpatialCoh.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/avg_spatial_coh.png"
view.main(scp_args.split())

## 2. Temporal Coherence

Temporal coherence is a measure of phase consistency for a pixel across the time series. A high level of temporal coherence indicates that phase changes in a pixel were steady, providing a measure of reliability. 

- [0, 1] range
- 0 = unreliable, 1 = reliable
- recommended threshold for dense time series: 0.7 (Yunjun et al, 2019)

In [ ]:
%matplotlib widget
scp_args = f"{mint_path}/temporalCoherence.h5 --dpi 600 --figsize 15 15 --outfile {plot_path}/temporal_coh.png"
view.main(scp_args.split())

---
## 3. Velocity error analysis

The estimated velocity also comes with an expression of unecrtainty which is simply based on the goodness of fit while fitting a linear model to the time-series. This quantity is saved in "velocity.h5" under the velocityStd dataset. 

- Velocity error increases with distance from the reference point
- Can indicate atmospheric delay

In [ ]:
%matplotlib widget
scp_args = f'{mint_path}/velocity.h5 velocityStd -v 0 1 --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity_err.png'
view.main(scp_args.split())

---
## 4. Compare InSAR time-series with GPS time-series in LOS direction

- http://geodesy.unr.edu/NGLStationPages/gpsnetmap/GPSNetMap.html
- http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt

**Access University of Nevada, Reno GPS station metadata and select a list of available stations** 

Ignore stations:
- outside the AOI
- outside the time range of the time series
- on no-data pixels

Two GPS stations are required for LOS conversion

In [ ]:
# Download DataHolding.txt and write to GPS_stations.csv
# in MintPy directory
gps.create_unr_gps_csv(mint_path)

# Finds stations in the AOI and time span of the time series.
# Ignore stations with no time series velocity data
gps_stations = gps.get_gps_stations(mint_path)
gps_stations

**Create interactive plot showing GPS station information**

Hover over GPS stations to view their details

In [ ]:
if gps_stations:
    gps_dict = gps.get_gps_dict(mint_path, gps_stations)
    velocity_png_pth = plot_path / "velocity.png"
    print("Hover over GPS stations to view their details")
    gps.gps_station_info_plot(mint_path, velocity_png_pth, gps_stations, gps_dict)


**Select a reference GPS station** 

In [ ]:
gps_station = widgets.RadioButtons(
    options=gps_stations,
    description='',
    disabled=False,
    layout=Layout(min_width='800px'))

print("Select a GPS station")
display(gps_station)

**Run the LOS comparison with the selected reference GPS station**

You can select different reference stations and rerun the following code cell multiple times to see results referenced to different areas of your time series. 

In [ ]:
%matplotlib widget

vmin, vmax = util.get_mintpy_vmin_vmax(mint_path/'velocity.h5')

scp_args = f"{mint_path}/velocity.h5 velocity -v {vmin} {vmax} --show-gps --ref-gps {gps_station.value} --gps-comp enu2los --gps-label --figsize 9 9"
with osl.work_dir(mint_path):
    view.main(scp_args.split())